In [1]:
import pandas as pd
import time
from sklearn.metrics import classification_report
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from pathlib import Path
import time
from tqdm import tqdm

In [2]:
# Settings
train_dir = Path('./data/train')
val_dir   = Path('./data/test')
img_size = 48
batch_size = 64

# Transforms
train_transforms = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(5),
    transforms.RandomAffine(degrees=0, shear=10, translate=(0.2, 0.2)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)  # normalize to [-1, 1]
])

val_transforms = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

# Datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transforms)
val_dataset   = datasets.ImageFolder(root=val_dir, transform=val_transforms)
print(f"Train images: {len(train_dataset)}")
print(f"Val images: {len(val_dataset)}")

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

# Class names
class_names = train_dataset.classes
print("Classes:", class_names)

Train images: 33595
Val images: 7178
Classes: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


In [3]:
class InvertedResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride, expand_ratio):
        super().__init__()
        self.use_res_connect = stride == 1 and in_channels == out_channels
        hidden_dim = in_channels * expand_ratio

        layers = []

        # Expansion
        if expand_ratio != 1:
            layers.extend([
                nn.Conv2d(in_channels, hidden_dim, kernel_size=1, bias=False),
                nn.BatchNorm2d(hidden_dim),
                nn.ReLU6(inplace=True)
            ])

        # Depthwise
        layers.extend([
            nn.Conv2d(hidden_dim, hidden_dim, kernel_size=3, stride=stride, padding=1, groups=hidden_dim, bias=False),
            nn.BatchNorm2d(hidden_dim),
            nn.ReLU6(inplace=True)
        ])

        # Projection
        layers.extend([
            nn.Conv2d(hidden_dim, out_channels, kernel_size=1, bias=False),
            nn.BatchNorm2d(out_channels)
        ])

        self.block = nn.Sequential(*layers)

    def forward(self, x):
        if self.use_res_connect:
            return x + self.block(x)
        else:
            return self.block(x)

class MobileNetV2_48x48(nn.Module):
    def __init__(self, num_classes=7, dropout_rate=0.0):
        super().__init__()
        self.dropout_rate = dropout_rate
        self.dropout = nn.Dropout(dropout_rate)

        # Initial convolution layer
        self.init_conv = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1, bias=False),  # 48x48 input
            nn.BatchNorm2d(32),
            nn.ReLU6(inplace=True)
        )

        # Bottleneck configuration: (t, c, n, s)
        bottlenecks_cfg = [
            (1, 16, 1, 1),
            (6, 24, 1, 1),  # no downsampling yet
            (6, 32, 2, 2),
            (6, 64, 1, 2),
            (6, 96, 1, 1),
        ]

        blocks = []
        in_channels = 32
        for t, out_channels, n, s in bottlenecks_cfg:
            for i in range(n):
                stride = s if i == 0 else 1
                blocks.append(InvertedResidualBlock(in_channels, out_channels, stride, t))
                in_channels = out_channels
        self.bottlenecks = nn.Sequential(*blocks)

        self.last_conv = nn.Sequential(
            nn.Conv2d(in_channels, 128, kernel_size=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU6(inplace=True),
            nn.Dropout(dropout_rate)
        )

        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.init_conv(x)
        x = self.bottlenecks(x)
        x = self.last_conv(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        return self.classifier(x)

In [4]:
# Define MobileNetV2 model
mobilenet_model = MobileNetV2_48x48(num_classes=len(class_names))
total_params = sum(p.numel() for p in mobilenet_model.parameters())
print(f"Total parameters: {total_params}")

Total parameters: 132935


In [ ]:
if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")
else:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device: {device}")
mobilenet_model = MobileNetV2_48x48(num_classes=len(class_names)).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(mobilenet_model.parameters(), lr=1e-3,  weight_decay=5e-5)

Using device: mps


In [6]:
def train(model, train_loader, val_loader, optimizer, criterion, num_epochs=15, model_name='mobv2_model.pth'):
    history = []  # collect metrics per epoch

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        start_time = time.time()

        # Training phase
        model.train()
        train_loss, correct, total = 0, 0, 0
        for images, labels in tqdm(train_loader, desc="Training"):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        train_acc = correct / total
        train_loss /= total

        # Validation phase
        model.eval()
        val_loss, correct, total = 0, 0, 0
        y_true, y_pred = [], []

        with torch.no_grad():
            for images, labels in tqdm(val_loader, desc="Validation"):
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * images.size(0)

                _, predicted = outputs.max(1)
                correct += (predicted == labels).sum().item()
                total += labels.size(0)

                y_true.extend(labels.cpu().numpy())
                y_pred.extend(predicted.cpu().numpy())

        val_acc = correct / total
        val_loss /= total

        # Classification report
        report = classification_report(y_true, y_pred, target_names=class_names, output_dict=True)
        val_f1_macro = report['macro avg']['f1-score']

        epoch_time = time.time() - start_time

        # Collect metrics
        epoch_data = {
            'epoch': epoch + 1,
            'train_loss': train_loss,
            'val_loss': val_loss,
            'train_acc': train_acc,
            'val_acc': val_acc,
            'val_f1_macro': val_f1_macro,
            'epoch_time_sec': epoch_time
        }

        # Add per-class metrics
        for cls in class_names:
            for metric in ['precision', 'recall', 'f1-score']:
                key = f'{cls}_{metric}'
                epoch_data[key] = report[cls][metric]

        history.append(epoch_data)

        # Console output
        print("\nValidation Report:")
        for cls in class_names:
            cls_metrics = report[cls]
            print(f"{cls}: Prec={cls_metrics['precision']:.3f} | Rec={cls_metrics['recall']:.3f} | F1={cls_metrics['f1-score']:.3f}")

        print(f"\nEpoch Summary: Train Loss={train_loss:.4f} | Val Loss={val_loss:.4f} | Train Acc={train_acc:.3f} | Val Acc={val_acc:.3f}")
        print(f"Epoch time: {epoch_time:.2f} seconds")

    torch.save(model.state_dict(), f"results/{model_name}")
    print(f"Saved model to 'results/{model_name}'")

    return history

In [ ]:
def build_model(dropout):
    return MobileNetV2_48x48(num_classes=len(class_names), dropout_rate=dropout).to(device)

# Define settings to test
configs = [
    # No dropout, no weight decay
    {'name': 'baseline_adam_drop0_wd0', 'optimizer': 'adam', 'lr': 1e-3, 'dropout': 0.0, 'weight_decay': 0.0},
    # Dropout only
    {'name': 'drop15_wd0', 'optimizer': 'adam', 'lr': 1e-3, 'dropout': 0.15, 'weight_decay': 0.0},
    # Weight decay only
    {'name': 'drop0_wd5e5', 'optimizer': 'adam', 'lr': 1e-3, 'dropout': 0.0, 'weight_decay': 5e-5},
    # Combined dropout + weight decay
    {'name': 'drop15_wd5e5', 'optimizer': 'adam', 'lr': 1e-3, 'dropout': 0.15, 'weight_decay': 5e-5},
    # Same but using SGD to test optimizer difference
    {'name': 'drop15_wd5e5_sgd', 'optimizer': 'sgd', 'lr': 1e-2, 'dropout': 0.15, 'weight_decay': 5e-5},
]

for config in configs:
    print(f"\nTraining config: {config['name']}")
    model = build_model(config['dropout'])

    if config['optimizer'] == 'adam':
        optimizer = optim.Adam(model.parameters(), lr=config['lr'], weight_decay=config['weight_decay'])
    else:
        optimizer = optim.SGD(model.parameters(), lr=config['lr'], momentum=0.9, weight_decay=config['weight_decay'])

    criterion = nn.CrossEntropyLoss()
    history = train(model, train_loader, val_loader, optimizer, criterion, num_epochs=15, model_name=config['name'])
    pd.DataFrame(history).to_csv(f'results/history_{config["name"]}.csv', index=False)


Training config: mobv2_adam_lr3_dropout0_bn

Epoch 1/15


Validation: 100%|██████████| 113/113 [00:20<00:00,  5.59it/s]



Validation Report:
angry: Prec=0.288 | Rec=0.033 | F1=0.060
disgust: Prec=0.015 | Rec=0.090 | F1=0.026
fear: Prec=0.231 | Rec=0.023 | F1=0.043
happy: Prec=0.392 | Rec=0.805 | F1=0.527
neutral: Prec=0.333 | Rec=0.079 | F1=0.128
sad: Prec=0.321 | Rec=0.381 | F1=0.348
surprise: Prec=0.555 | Rec=0.587 | F1=0.571

Epoch Summary: Train Loss=1.6617 | Val Loss=1.6650 | Train Acc=0.350 | Val Acc=0.356
Epoch time: 164.02 seconds

Epoch 2/15


Validation: 100%|██████████| 113/113 [00:19<00:00,  5.66it/s]



Validation Report:
angry: Prec=0.444 | Rec=0.013 | F1=0.024
disgust: Prec=0.017 | Rec=0.459 | F1=0.032
fear: Prec=0.304 | Rec=0.014 | F1=0.026
happy: Prec=0.794 | Rec=0.414 | F1=0.544
neutral: Prec=0.422 | Rec=0.257 | F1=0.320
sad: Prec=0.308 | Rec=0.330 | F1=0.319
surprise: Prec=0.507 | Rec=0.650 | F1=0.570

Epoch Summary: Train Loss=1.4176 | Val Loss=2.1831 | Train Acc=0.460 | Val Acc=0.290
Epoch time: 157.36 seconds

Epoch 3/15


...


Validation Report:
angry: Prec=0.239 | Rec=0.011 | F1=0.022
disgust: Prec=0.032 | Rec=0.153 | F1=0.053
fear: Prec=0.188 | Rec=0.246 | F1=0.213
happy: Prec=0.785 | Rec=0.573 | F1=0.663
neutral: Prec=0.483 | Rec=0.272 | F1=0.348
sad: Prec=0.332 | Rec=0.439 | F1=0.378
surprise: Prec=0.406 | Rec=0.792 | F1=0.537

Epoch Summary: Train Loss=1.4294 | Val Loss=1.5734 | Train Acc=0.455 | Val Acc=0.395
Epoch time: 176.85 seconds

Epoch 3/15


Validation: 100%|██████████| 113/113 [00:19<00:00,  5.77it/s]



Validation Report:
angry: Prec=0.305 | Rec=0.189 | F1=0.233
disgust: Prec=0.023 | Rec=0.586 | F1=0.044
fear: Prec=0.294 | Rec=0.034 | F1=0.061
happy: Prec=0.776 | Rec=0.446 | F1=0.567
neutral: Prec=0.582 | Rec=0.118 | F1=0.196
sad: Prec=0.473 | Rec=0.097 | F1=0.161
surprise: Prec=0.340 | Rec=0.859 | F1=0.488

Epoch Summary: Train Loss=1.3071 | Val Loss=2.6365 | Train Acc=0.506 | Val Acc=0.286
Epoch time: 167.29 seconds

Epoch 4/15


Training:   2%|▏         | 11/525 [00:06<05:26,  1.57it/s]

: 

In [ ]:
# Train best model for 40 epochs
best_dropout = 0.15  # CHANGE DEPENDING ON YOUR BEST CONFIG
best_lr = 1e-3
best_weight_decay = 5e-5
best_optimizer_name = 'adam'  # or 'sgd'

model = build_model(best_dropout)
if best_optimizer_name == 'adam':
    optimizer = optim.Adam(model.parameters(), lr=best_lr, weight_decay=best_weight_decay)
else:
    optimizer = optim.SGD(model.parameters(), lr=best_lr, momentum=0.9, weight_decay=best_weight_decay)
criterion = nn.CrossEntropyLoss()

history = train(model, train_loader, val_loader, optimizer, criterion, num_epochs=40, model_name='mobv2_final_model.pth')
pd.DataFrame(history).to_csv(f'results/history_mobv2_final_model.csv', index=False)
df_metrics = pd.DataFrame(history)
display(df_metrics.head())